In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [2]:
jobs_results = pd.read_csv("../Data/jobs_results.csv")
jobs_results.head()

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id,search_name,search_location,schedule_type
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,Language Fluency: English (Required)\nThe requ...,Deliver highly complex solutions with signific...,General Description of Available Benefits for ...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...,block-chain-1,USA,Full-time
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,NaN,NaN,NaN,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...,block-chain-1,USA,Contractor
2,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,Ability to parse and understand EVM (and/or So...,Tackling interesting problems - we are ingesti...,An accelerated growth trajectory - you will ha...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,block-chain-1,USA,Full-time
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",We’re looking for a capable generalist enginee...,You will be designing and building our core so...,"We’re wrapping up our presale for $JAM, so you...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,block-chain-1,USA,Full-time
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,NaN,NaN,NaN,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,block-chain-1,USA,Full-time


### Density of Data Science Jobs

In [3]:
search_for_coordinates = True # set to true to run the folium plot incl the coordinate search

In [4]:
jobs_results['location'].unique()

array(['Anywhere', 'New York, NY', 'Los Angeles, CA', 'Livermore, CA',
       'Irving, TX', 'Mechanicsburg, PA', 'Texas', 'Milwaukee, WI',
       'United States', 'Dallas, TX', 'Chicago, IL',
       'Colorado Springs, CO', 'Chevy Chase, MD', 'Montreal, MO',
       'Cupertino, CA', 'Lexington, MA', 'Seattle, WA',
       'San Francisco, CA', 'Washington', 'Texas   (+5 others)',
       'Waltham, MA', 'Redmond, WA', 'North Reading, MA', 'Palo Alto, CA',
       'Milpitas, CA', 'Crownsville, MD', 'Boston, MA', 'San Mateo, CA',
       'Maryland', 'Annapolis Junction, MD', 'Annapolis, MD',
       'Catonsville, MD', 'Columbia, MD', 'Olathe, KS', 'Bentonville, AR',
       'Missouri', 'Rogers, AR', 'Beatrice, NE', 'Redwood City, CA',
       'Laurel, MD', 'Plano, TX', 'Oak Ridge, TN',
       'Irving, TX  (+1 other)', 'Bolinas, CA', 'Oklahoma City, OK',
       'United States   (+26 others)', 'Los Gatos, CA', 'Dearing, KS',
       'Hanover, MD', 'Fort Meade, MD', 'South San Francisco, CA',
       'A

In [5]:
density_of_data_science_jobs = jobs_results.copy()

# remove rows where location is anywhere
density_of_data_science_jobs = density_of_data_science_jobs.loc[density_of_data_science_jobs['location'] != 'Anywhere']

# remove if parenthesis are present (these don't have city info)
density_of_data_science_jobs = density_of_data_science_jobs.loc[~density_of_data_science_jobs['location'].str.contains('\(')]

# remove if they don't have commas (these don't have cities either)
density_of_data_science_jobs = density_of_data_science_jobs.loc[density_of_data_science_jobs['location'].str.contains(',')]


density_of_data_science_jobs['location'].unique()

array(['New York, NY', 'Los Angeles, CA', 'Livermore, CA', 'Irving, TX',
       'Mechanicsburg, PA', 'Milwaukee, WI', 'Dallas, TX', 'Chicago, IL',
       'Colorado Springs, CO', 'Chevy Chase, MD', 'Montreal, MO',
       'Cupertino, CA', 'Lexington, MA', 'Seattle, WA',
       'San Francisco, CA', 'Waltham, MA', 'Redmond, WA',
       'North Reading, MA', 'Palo Alto, CA', 'Milpitas, CA',
       'Crownsville, MD', 'Boston, MA', 'San Mateo, CA',
       'Annapolis Junction, MD', 'Annapolis, MD', 'Catonsville, MD',
       'Columbia, MD', 'Olathe, KS', 'Bentonville, AR', 'Rogers, AR',
       'Beatrice, NE', 'Redwood City, CA', 'Laurel, MD', 'Plano, TX',
       'Oak Ridge, TN', 'Bolinas, CA', 'Oklahoma City, OK',
       'Los Gatos, CA', 'Dearing, KS', 'Hanover, MD', 'Fort Meade, MD',
       'South San Francisco, CA', 'Arlington, VA', 'Ann Arbor, MI',
       'San Diego, CA', 'Miami, FL', 'Sunnyvale, CA', 'Berkeley, CA',
       'Littleton, CO', 'Atlanta, GA', 'Silver Spring, MD',
       'West Col

In [6]:
state_abb_and_names = {

', AL': ', Alabama',
', AK': ', Alaska',
', AZ': ', Arizona',
', AR': ', Arkansas',
', AS': ', American Samoa',
', CA': ', California',
', CO': ', Colorado',
', CT': ', Connecticut',
', DE': ', Delaware',
', DC': ', District of Columbia',
', FL': ', Florida',
', GA': ', Georgia',
', GU': ', Guam',
', HI': ', Hawaii',
', ID': ', Idaho',
', IL': ', Illinois',
', IN': ', Indiana',
', IA': ', Iowa',
', KS': ', Kansas',
', KY': ', Kentucky',
', LA': ', Louisiana',
', ME': ', Maine',
', MD': ', Maryland',
', MA': ', Massachusetts',
', MI': ', Michigan',
', MN': ', Minnesota',
', MS': ', Mississippi',
', MO': ', Missouri',
', MT': ', Montana',
', NE': ', Nebraska',
', NV': ', Nevada',
', NH': ', New Hampshire',
', NJ': ', New Jersey',
', NM': ', New Mexico',
', NY': ', New York',
', NC': ', North Carolina',
', ND': ', North Dakota',
', MP': ', Northern Mariana Islan',
', OH': ', Ohio',
', OK': ', Oklahoma',
', OR': ', Oregon',
', PA': ', Pennsylvania',
', PR': ', Puerto Rico',
', RI': ', Rhode Island',
', SC': ', South Carolina',
', SD': ', South Dakota',
', TN': ', Tennessee',
', TX': ', Texas',
', TT': ', Trust Territories',
', UT': ', Utah',
', VT': ', Vermont',
', VA': ', Virginia',
', VI': ', Virgin Islands',
', WA': ', Washington',
', WV': ', West Virginia',
', WI': ', Wisconsin',
', WY': ', Wyoming'

}      	

In [7]:
# Replace state abbreviations with state names
density_of_data_science_jobs['location'] = density_of_data_science_jobs['location'].replace(state_abb_and_names, regex=True)
density_of_data_science_jobs['location'].unique()

array(['New York, New York', 'Los Angeles, California',
       'Livermore, California', 'Irving, Texas',
       'Mechanicsburg, Pennsylvania', 'Milwaukee, Wisconsin',
       'Dallas, Texas', 'Chicago, Illinois', 'Colorado Springs, Colorado',
       'Chevy Chase, Maryland', 'Montreal, Missouri',
       'Cupertino, California', 'Lexington, Massachusetts',
       'Seattle, Washington', 'San Francisco, California',
       'Waltham, Massachusetts', 'Redmond, Washington',
       'North Reading, Massachusetts', 'Palo Alto, California',
       'Milpitas, California', 'Crownsville, Maryland',
       'Boston, Massachusetts', 'San Mateo, California',
       'Annapolis Junction, Maryland', 'Annapolis, Maryland',
       'Catonsville, Maryland', 'Columbia, Maryland', 'Olathe, Kansas',
       'Bentonville, Arkansas', 'Rogers, Arkansas', 'Beatrice, Nebraska',
       'Redwood City, California', 'Laurel, Maryland', 'Plano, Texas',
       'Oak Ridge, Tennessee', 'Bolinas, California',
       'Oklahoma Ci

In [8]:
# create a counts table
location_counts = density_of_data_science_jobs['location'].value_counts().reset_index()
location_counts.columns = ['location', 'count']

display(location_counts)


,location,count
0,"New York, New York",44
1,"Washington, District of Columbia",44
2,"San Francisco, California",28
3,"Annapolis Junction, Maryland",21
4,"Arlington, Virginia",19
...,...,...
144,"Irvine, California",1
145,"Berkeley, California",1
146,"Natick, Massachusetts",1
147,"Arlington, Texas",1


In [9]:
if search_for_coordinates == True:

    # use geopy nominatim to retrive coordinates using location names https://geopy.readthedocs.io/en/stable/
    geolocator = Nominatim(user_agent="city_locator", timeout=10)
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

    location_counts['coordinate'] = location_counts['location'].apply(geocode)


    display(location_counts.head())

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Washington, District of Columbia',), **{}).
Traceback (most recent call last):
  File "/Users/victor/miniconda3/envs/spring2023env/lib/python3.10/site-packages/geopy/geocoders/base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "/Users/victor/miniconda3/envs/spring2023env/lib/python3.10/site-packages/geopy/adapters.py", line 447, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "/Users/victor/miniconda3/envs/spring2023env/lib/python3.10/site-packages/geopy/adapters.py", line 475, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 502

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/victor/miniconda3/envs/spring2023env/lib/python3.10/site-packages/geopy/extra/rate_limiter.py", line 136, in _retries_

In [12]:
if search_for_coordinates == True:

    for i, row in location_counts.iterrows():
        if row['coordinate']: # if row is not empty
            location_counts.loc[i, 'latitude'] = row['coordinate'].latitude
            location_counts.loc[i, 'longitude'] = row['coordinate'].longitude
        else:
            location_counts.loc[i, 'latitude'] = None
            location_counts.loc[i, 'longitude'] = None

# drop the coordinate column
location_counts = location_counts.drop('coordinate', axis=1)

# check
display(location_counts.head())

KeyError: 'coordinate'

In [22]:
if search_for_coordinates == True:

    m = folium.Map(location=[location_counts['latitude'].mean(), location_counts['longitude'].mean()], zoom_start=4)

    marker_cluster = MarkerCluster().add_to(m)

    for idx, row in location_counts.iterrows():
        folium.Marker(
            [row['latitude'], row['longitude']], 
            tooltip=row['location']
        ).add_to(marker_cluster)

    m.save('../Report/Outputs/folium_job_counts.html')
    m